In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go

In [ ]:
ticker = "spy"
data = yf.download(ticker, period="200d", interval="1d", progress=False)

In [121]:
def price_change(data):
    col_to_keep = list(data.columns)

    data["chg"] = data["Close"].diff()
    data["chg_abs"] = abs(data["chg"])
    data["chg_abs_10ma"] = data["chg_abs"].rolling(window=10).mean()
    data["chg_abs_10std"] = data["chg_abs"].rolling(window=10).std()
    data["chg_abs_10xtrm"] = data["chg_abs_10ma"] + data["chg_abs_10std"]
    data["chg_clean"] = np.where(data["chg_abs"] > data["chg_abs_10xtrm"], data["chg_abs_10xtrm"], data["chg_abs_10ma"])
    data["chg_clean_10ma"] = data["chg_clean"].rolling(window=10).mean()
    data["chg_clean_10std"] = data["chg_clean"].rolling(window=10).std()
    data["chg_clean_10xtrm"] = data["chg_clean_10ma"] + data["chg_clean_10std"]

    data["chg_flag"] = np.where(data["chg"] < -data["chg_clean_10xtrm"], True, False)
    col_to_keep.extend(["chg", "chg_abs_10xtrm", "chg_clean_10xtrm", "chg_flag"])
    # data = data[col_to_keep]
    
    return data

def local_minimum(data):
    col_to_keep = list(data.columns)
    data["5d_close_min"] = data["Close"].rolling(window=5).min()
    data["5d_open_min"] = data["Open"].rolling(window=5).min()
    data["5d_min"] = data[["5d_close_min", "5d_open_min"]].min(axis=1)
    data["min_flag"] = np.where(data["Close"] == data["5d_min"], True, False)
    return data

def local_maximum(data):
    col_to_keep = list(data.columns)
    data["5d_close_max"] = data["Close"].rolling(window=5).max()
    data["5d_open_max"] = data["Open"].rolling(window=5).max()
    data["5d_max"] = data[["5d_close_max", "5d_open_max"]].max(axis=1)
    data["max_flag"] = np.where(data["Close"] == data["5d_max"], True, False)
    return data



In [136]:
tickers = ["soxx"]

for ticker in tickers:
    data = yf.download(ticker, period="200d", interval="1d", progress=False)
    data = price_change(data)
    data = local_minimum(data)
    # data = local_maximum(data)

    data["risky"] = data[["min_flag", "chg_flag"]].sum(axis=1).rolling(window=5).sum()
    # data["recover"] = np.where(data["max_flag"], -1, 0)

    # data["risky"] -= data["max_flag"]

    fig = go.Figure(
        data=[
            go.Candlestick(
                x=data.index,
                open=data["Open"],
                high=data["High"],
                low=data["Low"],
                close=data["Close"],
            )
        ]
    )

    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data["Close"].rolling(window=10).mean(),
            mode="lines",
            name="10ma",
            marker_color="green",
            opacity=0.7,
        )
    )

    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data["Close"].rolling(window=15).mean(),
            mode="lines",
            name="15ma",
            marker_color="red",
            opacity=0.7,
        )
    )

    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=data["Close"].ewm(span=15).mean(),
            mode="lines",
            name="15ema",
            marker_color="black",
            opacity=0.7,
        )
    )

    flag_df = data[data["chg_flag"]]
    fig.add_trace(go.Scatter(
        x=flag_df.index,
        y=flag_df["High"] * 1.01,
        mode="markers",
        name="chg_flag",
        marker=dict(symbol="circle", color="red", size=10)
    ))

    flag_df = data[data["min_flag"]]
    fig.add_trace(go.Scatter(
        x=flag_df.index,
        y=flag_df["High"] * 1.02,
        mode="markers",
        name="min_flag",
        marker=dict(symbol="square", color="green", size=10)
    ))

    flag_df = data[data["risky"] > 1]
    fig.add_trace(go.Scatter(
        x=flag_df.index,
        y=flag_df["High"] * 1.0,
        mode="markers",
        name="risky",
        marker=dict(symbol="cross", color="black", size=10)
    ))

    # Update the layout
    fig.update_layout(
        title="Sample Candlestick Chart",
        yaxis_title="Stock Price",
        xaxis_title="Date",
        height=600,
        xaxis_rangeslider_visible=False,
        margin=dict(l=0, r=10, t=30, b=30),
    )

    # Show the figure
    fig.show()
